# Variables

In [1]:
objrecogImg1 = "Blured.jpeg"
objrecogImg2 = "IMG_2291.jpeg"

objectMask1 = "car_1.png"
objectMask2 = "car_4.png"

sktlimg1 = ""
sktlimg2masks = ["person_1.png","person_2.png"]

car =""
rndobj = ""

# 1.

# Instance Seg Out Std

In [2]:
# Package Import

from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2 import model_zoo

import cv2
import json
import numpy as np
from PIL import Image

### --- Code From : https://www.youtube.com/watch?v=Pb3opEFP94U --- ###

class Detector:
    def __init__(self,model_type):
        self.cfg = get_cfg()
        self.moodel_type = model_type
        
    
        if model_type == "OD":
            self.cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
            self.cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
        elif model_type == "IS":
            self.cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
            self.cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
        elif model_type == "PS":
            self.cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
            self.cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")

        self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
        self.cfg.MODEL.DEVICE = "cpu"

        self.predictor = DefaultPredictor(self.cfg)

### --- Code From : https://www.youtube.com/watch?v=Pb3opEFP94U --- ###
        
# Code below is based on X 
        
    #Perform object detection/segmentation on image
    def onImage(self, imagePath,masks_path):
        #Read Image to np array
        image = cv2.imread(imagePath)
        #If model type is panoptic segementation
        if self.moodel_type != "PS":
            # Create predictions for specifed image
            predictions = self.predictor(image)
            
            #Initializes a visualizer with BGR converted image, dataset metadata, and sets instance visualization mode to black and white
            viz = Visualizer(image[:,:,::-1], metadata = MetadataCatalog.get(self.cfg.DATASETS.TRAIN[0]),
            instance_mode = ColorMode.IMAGE_BW)

            #Get models predictions, using CPU.
            direct_outputs = predictions["instances"].to("cpu")

            #Draws model's predictions on the image.
            output = viz.draw_instance_predictions(direct_outputs)

            #Get the metadata catalog for this configuration
            metadata = MetadataCatalog.get(self.cfg.DATASETS.TRAIN[0])
            #Get the class list for 'things'
            class_names = metadata.thing_classes

            #Create an array for the data to be written to the json
            data_to_save = []
            #For each of the outputs (predictions)
            for i in range(len(direct_outputs)):
                #Create a predictions dict with the bounding box, label and score
                pred = {
                    "bbox": direct_outputs.pred_boxes.tensor.numpy()[i].tolist(),
                    "label": class_names[direct_outputs.pred_classes[i].item()],
                    "score": direct_outputs.scores[i].item(),
                }
                #Appepnd the predictions to the array
                data_to_save.append(pred)

        # If model selected is not panoptic segmentation
        else:
            # Create predictions for specifed image, using panoptic segmenation
            predictions, segementInfo = self.predictor(image)["panoptic_seg"]
            # Initializes a visualizer with BGR converted imagea and dataset metadata
            viz = Visualizer(image[:,:,::-1],MetadataCatalog.get(self.cfg.DATASETS.TRAIN[0]))
            # Draws model's predictions on the image.
            output = viz.draw_panoptic_seg_predictions(predictions.to("cpu"),segementInfo)
            # Get the metadata catalog for this configuration
            metadata = MetadataCatalog.get(self.cfg.DATASETS.TRAIN[0])
            # Get the class list for 'things'
            thing_classes = metadata.thing_classes
            # Get the class list for 'stuff'
            stuff_classes = metadata.stuff_classes

            #Create an array for the data to be written to the json
            data_to_save = []

            # For each of the segements detected
            for segment_info in segementInfo:
                #If the current segement is a part of the 'thing' class
                if segment_info["isthing"]:  # Check if the segment is a "thing"
                    # ID of the the current segement from the 'thing' class
                    category_id = segment_info["category_id"]
                    # Human label for the current 'thing' i.e person
                    label = thing_classes[category_id] 
                    # Unqiue id for this 'stuff'
                    segment_id = segment_info["id"]


                    # Create a mask for the current segment
                    mask = predictions == segment_id
                    mask_np = np.array(mask.cpu(), dtype=np.uint8)  # Mask with 1s for the object

                    # Find contours in the mask
                    contours, _ = cv2.findContours(mask_np, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                    for cnt in contours:
                        # Compute the bounding box for the contour
                        x, y, w, h = cv2.boundingRect(cnt)
                        
                        # Bounding box as (x, y, w, h)
                        bbox = [x, y, x+w, y+h]  # Format: [x_min, y_min, x_max, y_max]

                    # Use the mask to extract the object from the original image
                    # Converting to rgb since that was the orginal
                    orignal_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    #Create the mask image, removing all pixels 
                    object_image = cv2.bitwise_and(orignal_image, orignal_image, mask=mask_np)

                    # Save the extracted object as an image
                    object_image_pil = Image.fromarray(object_image)
                    object_image_filename = f"{label}_{segment_id}.png"

                    # Save the data including the object image filename
                    data = {
                        "label": label,
                        "image": object_image_filename,
                        "bbox": bbox
                    }
                    data_to_save.append(data)
                #The current segement must be a part of the 'stuff' class
                else:
                    # ID of the the current segement from the 'thing' class
                    category_id = segment_info["category_id"]
                    # Human label for the current 'stuff' i.e sky
                    label = stuff_classes[category_id] 
                    # Unqiue id for this 'thing'
                    segment_id = segment_info["id"]

                    # Create a mask for the current segment
                    mask = predictions == segment_id
                    mask_np = np.array(mask.cpu(), dtype=np.uint8)  # Mask with 1s for the object

                    # Use the mask to extract the object from the original image
                    # Converting to rgb since that was the orginal
                    orignal_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    #Create the mask image, removing all pixels 
                    object_image = cv2.bitwise_and(orignal_image, orignal_image, mask=mask_np)

                    # Save the extracted object as an image
                    object_image_pil = Image.fromarray(object_image)
                    object_image_filename = f"{label}_{segment_id}.png"

                    # Save the data including the object image filename
                    data = {
                        "label": label,
                        "image": object_image_filename,
                        "bbox" : [-1]
                    }
                    data_to_save.append(data)

            #Order the data by the x values in the bounding boxes
            data_to_save = sorted(data_to_save,key=lambda item: item['bbox'][0])


            #Create an id for each of the entries with a bbox 
            counter = 0
            for i in data_to_save:
                if i['bbox'][0] == -1:
                    i["idNo"] = -1
                else:
                    i["idNo"] = counter
                    counter += 1

        # # Display the results
        cv2.namedWindow('Result', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('Result', 600, 400)
        cv2.imshow("Result", output.get_image()[:,:,::-1])
        cv2.waitKey(0)   
        cv2.destroyAllWindows()  

c:\Users\Ossia\Documents\SCCUIF\env\SCCUIF\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [ ]:
detectorIS = Detector(model_type="IS")
detectorIS.onImage(objrecogImg1,"")


The checkpoint state_dict contains keys that are not used by the model:
  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}
c:\Users\Ossia\Documents\SCCUIF\env\SCCUIF\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


# Text output of colour score in is vs ps

In [ ]:
detectorPS = Detector(model_type="PS")


The checkpoint state_dict contains keys that are not used by the model:
  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


# Std ps outut inc json

In [ ]:
detectorPS.onImage(objrecogImg1,"")

# Obj placement output 

In [ ]:
def get_obj_placment(df,x):
    if(x == 0):
        return (0,(df.iloc[x]['label']),(df.iloc[x+1]['label']))
    elif (x == len(df)-1):
        return ((df.iloc[x-1]['label']),(df.iloc[x]['label']),0)
    else:
        return (df.iloc[x-1]['label'],(df.iloc[x]['label']),(df.iloc[x+1]['label']))
    
# Run this in the alog notebook

# 2.

# Sktl with normal vs masks

# Sklt std upper and lowerbody outptut

In [ ]:
# Comp folder

# Colour output

In [ ]:
# Get from mainimport cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os
import pandas as pd
from skimage import io
from skimage.color import rgb2lab,deltaE_cie76,deltaE_ciede2000,deltaE_cmc

class HistColour:
    def __init__ (self,image_path):
        image = cv2.imread(image_path)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        #calculating histograms for each channel
        self.hue = cv2.calcHist([image],[0], None, [255], [0,255])
        self.sat = cv2.calcHist([image],[1], None, [255], [0,255])
        self.value = cv2.calcHist([image],[2], None, [255], [0,255])

class HistColourPercentage:
    def __init__ (self,image_path):
        image = cv2.imread(image_path)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # Calculate the total number of pixels
        total_pixels = image.shape[0] * image.shape[1]

        hue_value = cv2.calcHist([image],[0], None, [255], [0,255])
        sat_value = cv2.calcHist([image],[0], None, [255], [0,255])
        val_value = cv2.calcHist([image],[0], None, [255], [0,255])

        #calculating histograms for each channel
        self.hue = (hue_value/total_pixels) * 100
        self.sat = (sat_value/total_pixels) * 100
        self.value = (val_value/total_pixels) * 100




def get_distance(image1Name, image2Name):
    hist1 = HistColourPercentage(image1Name)
    hist2 = HistColourPercentage(image2Name)

    hue = np.subtract(hist1.hue,hist2.hue)
    sat = np.subtract(hist1.sat,hist2.sat)
    value = np.subtract(hist1.value,hist2.value)

    hueSquare = np.square(hue)
    satSquare = np.square(sat)
    valueSquare = np.square(value)

    totalAdd = np.add(np.add(hueSquare,satSquare),valueSquare)

    return np.average(np.sqrt(totalAdd))



def hist_color_intersect(image1Name, image2Name):
    img1 = cv2.imread(image1Name)
    img2 = cv2.imread(image2Name)
    # Convert images to HSV color space
    image1_hsv = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV)
    image2_hsv = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)

    # Calculate histograms
    hist1 = cv2.calcHist([image1_hsv],[0], None, [255], [0,255])
    hist2 = cv2.calcHist([image2_hsv],[0], None, [255], [0,255])

    # Normalize histograms
    cv2.normalize(hist1, hist1, 0, 1, cv2.NORM_MINMAX)
    cv2.normalize(hist2, hist2, 0, 1, cv2.NORM_MINMAX)

    hist1 = hist1 / np.sum(hist1)
    hist2 = hist2 / np.sum(hist2)
    # Compute histogram intersection
    intersection = cv2.compareHist(hist1, hist2, cv2.HISTCMP_INTERSECT)

    return intersection

def lab_space_comp(img1Name,img2Name):
    lab_data = []
    img1 = rgb2lab(io.imread(img1Name))
    img2 = rgb2lab(io.imread(img2Name))

    if img1.shape != img2.shape:
        img2 = cv2.resize(rgb2lab(io.imread(img2Name)), (img1.shape[:2][1],img1.shape[:2][0]))

    #Choose function based on label 
        # 1: Car
        if img1Name.split('_')[0] == "car" and img2Name.split('_')[0] == "car":
            #Show this works better on cars
            lab_data = deltaE_ciede2000(img1,img2)
        # 2: Person
        elif img1Name.split('_')[0] == "person" and img2Name.split('_')[0] == "person":
            #Show this works better on clothes
            lab_data = deltaE_cmc(img1,img2)
        # 3: Default
        else:
            lab_data = deltaE_cie76(img1,img2)

        return np.average(lab_data)
    

def lab_space_comp_percentage(img1Name,img2Name):
    lab_data = []
    img1 = rgb2lab(io.imread(img1Name))
    img2 = rgb2lab(io.imread(img2Name))

    if img1.shape != img2.shape:
        img2 = cv2.resize(rgb2lab(io.imread(img2Name)), (img1.shape[:2][1],img1.shape[:2][0]))

    img1 = cv2.normalize(img1, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    img2 = cv2.normalize(img2, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    #Choose function based on label 
    # 1: Car
    if img1Name.split('_')[0] == "car" and img2Name.split('_')[0] == "car":
        lab_data = deltaE_ciede2000(img1,img2)
    # 2: Person
    elif img1Name.split('_')[0] == "person" and img2Name.split('_')[0] == "person":
        lab_data = deltaE_cmc(img1,img2)
    # 3: Default
    else:
        lab_data = deltaE_cie76(img1,img2)

    return np.average(lab_data)


    

def img_2_img(src,ogImg1Name,ogImg2Name):
    img1Name = src + ogImg1Name
    img2Name = src + ogImg2Name
    img_2_img_comp_data = {
       #----Metric 1: Basic Distance 
        "comp" : ("src= " + src+ " : " + img1Name + "-" + img2Name),
        "basic_distance" :  str(get_distance(img1Name,img2Name)) + "%",
        "hist_intersection" :hist_color_intersect(img1Name,img2Name),
        "lab_space" : lab_space_comp(img1Name,img2Name)
    }
    
    return img_2_img_comp_data


def img_2_all(src,img1Name,imgList):
    img2_all_comp_data = {
        "overall_comp" : (src + ":" + img1Name),
        "list_comp" :[]
    }
    list_img_comp = []
    for i in imgList:
        list_img_comp.appennd(img_2_img(src,img1Name,i))

    img2_all_comp_data["list_comp"] = list_img_comp

    return img2_all_comp_data
    







# Lab space with and w/o specialistaion

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os
import pandas as pd
from skimage import io
from skimage.color import rgb2lab,deltaE_cie76,deltaE_ciede2000,deltaE_cmc

def lab_space_comp_percentage_no_spec(img1Name,img2Name):
    lab_data = []
    img1 = rgb2lab(io.imread(img1Name))
    img2 = rgb2lab(io.imread(img2Name))

    if img1.shape != img2.shape:
        img2 = cv2.resize(rgb2lab(io.imread(img2Name)), (img1.shape[:2][1],img1.shape[:2][0]))

    img1 = cv2.normalize(img1, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    img2 = cv2.normalize(img2, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    lab_data = deltaE_cie76(img1,img2)

    return np.average(lab_data)

In [ ]:
lab_space_comp_percentage_no_spec(objectMask1,objectMask2)

0.1197657

In [ ]:
lab_space_comp_percentage(objectMask1,objectMask2)

0.12528707

In [ ]:
get_distance(objectMask1,objectMask2)

0.02167713754441193

In [ ]:
hist_color_intersect(objectMask1,objectMask2)

0.9840429269852962

In [ ]:
lab_space_comp_percentage_no_spec(sktlimg2masks[0],sktlimg2masks[1])


0.3292964

In [ ]:
lab_space_comp_percentage(sktlimg2masks[0],sktlimg2masks[1])

0.5510889

# Df EXAMPLE OF Output 

In [ ]:
#Get from algoTest

##### ---------

# Sitting and standing output